In [90]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

In [45]:
# Good data
df = pd.read_csv('./data/kokil dec 6 reprepare/affcon_final_with_linguistics.csv')

In [71]:
# Messy data
df = pd.read_csv('./data/affcon_final_politeness_strategies_merge.csv')

In [60]:
df = df.dropna()

In [106]:
train, test = train_test_split(df, test_size=0.2)

y_train_deception = to_categorical(train['Input.deception_quadrant_cat'].tolist())
y_train_rapport_c = to_categorical(train['affcon_rapport'].tolist())
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

y_test_deception = to_categorical(test['Input.deception_quadrant_cat'].tolist())
y_test_rapport_c = to_categorical(test['affcon_rapport'].tolist())
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

X_train = train.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

X_test = test.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

### Individual classifiers are Random Forests

In [83]:
# Game move classifier
clf_gamemove = RandomForestClassifier(max_depth=2, random_state=0)
clf_gamemove.fit(X_train, y_train_gamemove)
y_pred_gamemove = clf_gamemove.predict(X_train)
y_pred_test_gamemove = clf_gamemove.predict(X_test)
print(classification_report(y_pred_gamemove, y_train_gamemove))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.84      0.91      3210

    accuracy                           0.84      3210
   macro avg       0.50      0.42      0.46      3210
weighted avg       1.00      0.84      0.91      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [107]:
# Rapport classifier
clf_rapport = RandomForestClassifier(max_depth=2, random_state=0)
clf_rapport.fit(X_train, y_train_rapport)
y_pred_rapport = clf_rapport.predict(X_train)
y_pred_test_rapport = clf_rapport.predict(X_test)
print(classification_report(y_pred_rapport, y_train_rapport))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.83      0.91      3210

    accuracy                           0.83      3210
   macro avg       0.50      0.42      0.45      3210
weighted avg       1.00      0.83      0.91      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
# Share Information classifier
clf_shareinfo = RandomForestClassifier(max_depth=2, random_state=0)
clf_shareinfo.fit(X_train, y_train_share_information)
y_pred_shareinfo = clf_shareinfo.predict(X_train)
y_pred_test_shareinfo = clf_shareinfo.predict(X_test)
print(classification_report(y_pred_shareinfo, y_train_share_information))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.81      0.89      3210

    accuracy                           0.81      3210
   macro avg       0.50      0.40      0.45      3210
weighted avg       1.00      0.81      0.89      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
# Reasoning classifier
clf_reasoning = RandomForestClassifier(max_depth=2, random_state=0)
clf_reasoning.fit(X_train, y_train_reasoning)
y_pred_reasoning = clf_reasoning.predict(X_train)
y_pred_test_reasoning = clf_reasoning.predict(X_test)
print(classification_report(y_pred_reasoning, y_train_reasoning))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.80      0.89      3210

    accuracy                           0.80      3210
   macro avg       0.50      0.40      0.44      3210
weighted avg       1.00      0.80      0.89      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
pred_df_arr = []
for i in range(0, len(y_pred_gamemove)):
    pred_obj = {}
    pred_obj['gamemove'] = y_pred_gamemove[i]
    pred_obj['reasoning'] = y_pred_reasoning[i]
    pred_obj['shareinfo'] = y_pred_shareinfo[i]
    pred_obj['rapport'] = y_pred_rapport[i]
    
    pred_df_arr.append(pred_obj)
    
pred_df = pd.DataFrame(pred_df_arr)

In [110]:
pred_test_df_arr = []
for i in range(0, len(y_pred_test_gamemove)):
    pred_obj = {}
    pred_obj['gamemove'] = y_pred_test_gamemove[i]
    pred_obj['reasoning'] = y_pred_test_reasoning[i]
    pred_obj['shareinfo'] = y_pred_test_shareinfo[i]
    pred_obj['rapport'] = y_pred_test_rapport[i]
    
    pred_test_df_arr.append(pred_obj)
    
pred_test_df = pd.DataFrame(pred_test_df_arr)

In [51]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [112]:
# Combined model

inputB = Input(shape=(pred_df.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
m = Model(inputs=inputB, outputs=c)

m.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m, precision_m,recall_m])
history = m.fit(x=pred_df, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df,y_test_deception), callbacks=[callback])

Epoch 1/32
51/51 [==============================] - 0s 8ms/step - loss: 7.7125 - acc: 0.0458 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.0523 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/32
51/51 [==============================] - 0s 4ms/step - loss: 7.7125 - acc: 0.0458 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.0523 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00


### Individual classifiers are LSTMs

Merge classifiers by Add

In [72]:
train, test = train_test_split(df, test_size=0.2)

y_train_deception = train['Input.deception_quadrant_cat'].tolist()
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

y_test_deception = test['Input.deception_quadrant_cat'].tolist()
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

X_train_col = train['Input.full_text']

X_test_col = test['Input.full_text']

In [73]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [74]:
le = LabelEncoder()

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [75]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

tok.fit_on_texts(X_test_col)
X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

In [76]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)
def create_lstm():
    Inp = Input(name='inputs', shape=[max_len])
    x = Embedding(max_words, 50, input_length=max_len)(Inp)
    x = LSTM(64, name='LSTM_01')(x)
    x = Dropout(0.5, name='Dropout')(x)
    x = Dense(128, activation='relu',name='Dense_01')(x)
    # x = Dropout(0.5,name='Dropout')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp, outputs=out)
    
    return model

In [77]:
# Rapport model
rapport_model = create_lstm()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
rapport_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_rapport), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 41ms/step - loss: 0.6686 - accuracy: 0.6025 - f1_m: 0.7440 - recall_m: 0.9723 - precision_m: 0.6062 - val_loss: 0.6576 - val_accuracy: 0.6144 - val_f1_m: 0.7531 - val_recall_m: 0.9658 - val_precision_m: 0.6184
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.6498 - accuracy: 0.6048 - f1_m: 0.7268 - recall_m: 0.8851 - precision_m: 0.6205 - val_loss: 0.6576 - val_accuracy: 0.6064 - val_f1_m: 0.7285 - val_recall_m: 0.8664 - val_precision_m: 0.6298


In [78]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

In [79]:
# Game move model
gamemove_model = create_lstm()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
gamemove_model.fit(X_train,y_train_gamemove,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_gamemove), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.6042 - accuracy: 0.7145 - f1_m: 0.8299 - recall_m: 0.9918 - precision_m: 0.7178 - val_loss: 0.5934 - val_accuracy: 0.7217 - val_f1_m: 0.8374 - val_recall_m: 1.0000 - val_precision_m: 0.7211
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5864 - accuracy: 0.7184 - f1_m: 0.8359 - recall_m: 0.9999 - precision_m: 0.7190 - val_loss: 0.5876 - val_accuracy: 0.7217 - val_f1_m: 0.8374 - val_recall_m: 1.0000 - val_precision_m: 0.7211
Epoch 3/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5736 - accuracy: 0.7186 - f1_m: 0.8348 - recall_m: 0.9971 - precision_m: 0.7189 - val_loss: 0.5976 - val_accuracy: 0.7205 - val_f1_m: 0.8365 - val_recall_m: 0.9970 - val_precision_m: 0.7214


In [80]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

In [81]:
# Reasoning model
reasoning_model = create_lstm()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
reasoning_model.fit(X_train,y_train_reasoning,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_reasoning), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.5558 - accuracy: 0.7617 - f1_m: 0.8634 - recall_m: 0.9964 - precision_m: 0.7628 - val_loss: 0.5378 - val_accuracy: 0.7691 - val_f1_m: 0.8692 - val_recall_m: 1.0000 - val_precision_m: 0.7696
Epoch 2/15
99/99 [==============================] - 3s 34ms/step - loss: 0.5292 - accuracy: 0.7639 - f1_m: 0.8649 - recall_m: 0.9973 - precision_m: 0.7645 - val_loss: 0.5327 - val_accuracy: 0.7668 - val_f1_m: 0.8651 - val_recall_m: 0.9746 - val_precision_m: 0.7787
Epoch 3/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5187 - accuracy: 0.7657 - f1_m: 0.8644 - recall_m: 0.9835 - precision_m: 0.7719 - val_loss: 0.5363 - val_accuracy: 0.7662 - val_f1_m: 0.8657 - val_recall_m: 0.9819 - val_precision_m: 0.7750


In [82]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

In [83]:
# Share Information model
shareinfo_model = create_lstm()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
shareinfo_model.fit(X_train,y_train_share_information,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_share_information), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.6837 - accuracy: 0.5443 - f1_m: 0.6133 - recall_m: 0.7571 - precision_m: 0.5587 - val_loss: 0.6747 - val_accuracy: 0.5750 - val_f1_m: 0.6293 - val_recall_m: 0.7003 - val_precision_m: 0.5737
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.6482 - accuracy: 0.6186 - f1_m: 0.6061 - recall_m: 0.5805 - precision_m: 0.6616 - val_loss: 0.6888 - val_accuracy: 0.5581 - val_f1_m: 0.4343 - val_recall_m: 0.3307 - val_precision_m: 0.6425


In [84]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

In [85]:
# Deception model
deception_model = create_lstm()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
deception_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_deception), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.2242 - accuracy: 0.9461 - f1_m: 0.9718 - recall_m: 0.9964 - precision_m: 0.9492 - val_loss: 0.2178 - val_accuracy: 0.9476 - val_f1_m: 0.9731 - val_recall_m: 1.0000 - val_precision_m: 0.9478
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.2033 - accuracy: 0.9494 - f1_m: 0.9741 - recall_m: 1.0000 - precision_m: 0.9496 - val_loss: 0.2062 - val_accuracy: 0.9476 - val_f1_m: 0.9731 - val_recall_m: 1.0000 - val_precision_m: 0.9478
Epoch 3/15
99/99 [==============================] - 3s 35ms/step - loss: 0.1906 - accuracy: 0.9494 - f1_m: 0.9741 - recall_m: 1.0000 - precision_m: 0.9496 - val_loss: 0.2252 - val_accuracy: 0.9476 - val_f1_m: 0.9731 - val_recall_m: 1.0000 - val_precision_m: 0.9478


In [86]:
# One hot encoding
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj = {}
    pred_obj['gamemove'] = gamemove_pred[i][0]
    pred_obj['reasoning'] = reasoning_pred[i][0]
    pred_obj['shareinfo'] = shareinfo_pred[i][0]
    #pred_obj['rapport'] = rapport_pred[i][0]
    
    pred_df_arr.append(pred_obj)
    
pred_df = pd.DataFrame(pred_df_arr)

In [87]:
pred_test_df_arr = []
for i in range(0, len(gamemove_pred_test)):
    pred_obj = {}
    pred_obj['gamemove'] = gamemove_pred_test[i][0]
    pred_obj['reasoning'] = reasoning_pred_test[i][0]
    pred_obj['shareinfo'] = shareinfo_pred_test[i][0]
    #pred_obj['rapport'] = rapport_pred_test[i][0]
    
    pred_test_df_arr.append(pred_obj)
    
pred_test_df = pd.DataFrame(pred_test_df_arr)

In [112]:
pred_df.head()

,gamemove,reasoning,shareinfo,rapport
0,0.709489,0.840170,0.837677,0.639576
1,0.683541,0.853558,0.680558,0.623211
2,0.577929,0.632894,0.490597,0.564213
3,0.711058,0.816315,0.778096,0.708829
4,0.618718,0.720346,0.556693,0.500282


In [113]:
from sklearn.utils import class_weight
class_weight = {'gamemove': 50,
                'reasoning': 50.,
                'shareinfo': 10, 'rapport': 2}

In [92]:
# Joint Model with one hot encoding 
inputB = Input(shape=(pred_df.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='softmax')(c)
full_model = Model(inputs=inputB, outputs=c)

full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
# history = full_model.fit(x=pred_df, y=y_train_deception, epochs=32, 
#                     batch_size=64, 
#                     validation_split=0.2, callbacks=[callback])

history = full_model.fit(x=pred_df, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])

Epoch 1/32
158/158 [==============================] - 1s 5ms/step - loss: 0.6932 - acc: 0.4398 - f1_m: 0.7494 - precision_m: 0.6046 - recall_m: 0.9949 - val_loss: 0.6931 - val_acc: 0.4114 - val_f1_m: 0.7397 - val_precision_m: 0.5901 - val_recall_m: 1.0000
Epoch 2/32
158/158 [==============================] - 1s 3ms/step - loss: 0.6931 - acc: 0.4584 - f1_m: 0.7525 - precision_m: 0.6054 - recall_m: 1.0000 - val_loss: 0.6931 - val_acc: 0.5886 - val_f1_m: 0.7397 - val_precision_m: 0.5901 - val_recall_m: 1.0000
Epoch 3/32
158/158 [==============================] - 1s 3ms/step - loss: 0.6931 - acc: 0.4582 - f1_m: 0.7478 - precision_m: 0.6020 - recall_m: 0.9937 - val_loss: 0.6931 - val_acc: 0.4114 - val_f1_m: 0.7397 - val_precision_m: 0.5901 - val_recall_m: 1.0000
Epoch 4/32
158/158 [==============================] - 1s 3ms/step - loss: 0.6931 - acc: 0.4813 - f1_m: 0.7523 - precision_m: 0.6054 - recall_m: 1.0000 - val_loss: 0.6931 - val_acc: 0.4110 - val_f1_m: 0.7397 - val_precision_m: 0.5901

In [91]:
# Joint Model with model Add
commonInput = Input(shape=[max_len])

out1 = gamemove_model(commonInput)    
out2 = reasoning_model(commonInput)   
out3 = shareinfo_model(commonInput)
out4 = rapport_model(commonInput)
#mergedOut = Add()([out1,out2,out3,out4])
mergedOut = Concatenate()([out1,out2,out3])

mergedOut = Flatten()(mergedOut)    
mergedOut = Dense(256, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(128, activation='relu')(mergedOut)
mergedOut = Dropout(.35)(mergedOut)
mergedOut = Dense(2, activation='softmax')(mergedOut)  #Cuz binary

mergedModel = Model(commonInput, mergedOut)
mergedModel.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', f1_m, recall_m, precision_m])
# mergedModel.fit(X_train,y_train_deception,
#           batch_size=128,
#           epochs=15,
#           validation_split=0.2, callbacks=[early_stop])

mergedModel.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_split=0.2, callbacks=[early_stop])

Epoch 1/15
79/79 [==============================] - 8s 98ms/step - loss: 0.6931 - accuracy: 0.5005 - f1_m: 0.7524 - recall_m: 1.0000 - precision_m: 0.6047 - val_loss: 0.6931 - val_accuracy: 0.4114 - val_f1_m: 0.7397 - val_recall_m: 1.0000 - val_precision_m: 0.5884
Epoch 2/15
79/79 [==============================] - 7s 86ms/step - loss: 0.6931 - accuracy: 0.5139 - f1_m: 0.7531 - recall_m: 0.9998 - precision_m: 0.6052 - val_loss: 0.6931 - val_accuracy: 0.4114 - val_f1_m: 0.7397 - val_recall_m: 1.0000 - val_precision_m: 0.5884
